# Chapter 5
# Numerical Integration

In many computational economic applications, one must compute the definite integral
of a real-valued function $u(y)$ with respect to a "weighting" function $f(y)$ over an interval
$I$ of $R^n$:

$$\int_I f(y)u(y) dy$$


The weighting function may be the identity function, $f(y) = 1$, in which case the integral represents
the area under the function $u(y)$. In other applications, $f(y)$ may be the probability
density of a random variable $\tilde y$, in which case the integral represents the expectation
of $u( \tilde y)$ when $I$ is the whole support of $\tilde y$.





In this notebook, we discuss three classes of numerical integration or numerical
quadrature methods<sup>1</sup>. All methods approximate the integral with a weighted sum of
function values:

$$\int_I  f(y)u(y)dy \approx \sum_{i=0}^{n} f_i u(y_i)\thinspace .$$

<sup>1</sup>Quadrature is a historical mathematical term that means calculating an area.

The methods differ only in how the *quadrature weights* $f_i$ and the *quadrature nodes*
$y_i$ are chosen.

1. Newton-Cotes rules employ piecewise polynomial approximations to the integrand
2. Gaussian quadrature methods employ nodes and weights that satisfy moment matching conditions
3. Monte Carlo methods employ equally weighted “random” nodes

**Newton-Cotes** methods approximate the integrand $u$ between nodes
using low order polynomials, and sum the integrals of the polynomials to approximate
the integral of $u$. Newton-Cotes methods are easy to implement, but are not particularly
efficient for computing the integral of a smooth function.

**Gaussian quadrature**
methods choose the nodes and weights to satisfy moment matching conditions, and
are more powerful than Newton-Cotes methods if the integrand is smooth.

**Monte Carlo and quasi-Monte Carlo integration** methods use "random" or "equidistributed"
nodes, and are simple to implement and are especially useful if the integration domain
is of high dimension or irregularly shaped.

## 5.2 Gaussian Quadrature
Gaussian quadrature rules are constructed with respect to specific weighting functions.


Specifically, for a weighting function $f(y)$ defined on an interval $I \in R$ of the real
line (for a given order of approximation $n$), the quadrature nodes $y_1, y_2, ..., y_n$
and quadrature weights $f_1, f_2, ..., f_n$ are chosen so as to satisfy the **$2n$ "moment-matching"
conditions:**




$$\int_I f(y)y^k dy = \sum_{i=1}^{n} f_i y_i^k\thinspace \forall \thinspace k = 0,...,2n-1. $$



The approximation to the integral is then computed by forming the prescribed weighted sum of
function values at the prescribed nodes:

$$\int_I f(y)u(y)  \thinspace dy \approx \sum_{i=1}^{n} f_i u(y_i)\thinspace .$$

By construction, an $n$-point Gaussian quadrature rule is order $2n - 1$ exact. If not
for rounding error, it will exactly compute the integral of any polynomial of order $2n - 1$ 
or less with respect to the weight function. Thus, if $u(y)$ can be accurately approximated by a
polynomial, Gaussian quadrature will provide a precise approximation to the integral.



![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/Comparison_Gaussquad_trapezoidal.svg/880px-Comparison_Gaussquad_trapezoidal.svg.png)



*Comparison between 2-point Gaussian and trapezoidal quadrature. The blue line is the polynomial ${\displaystyle u(y)=7y^{3}-8y^{2}-3y+3}$ , whose integral over [−1, 1] is 2/3. The trapezoidal rule returns the integral of the orange dashed line, equal to ${\displaystyle u(-1)+u(1)=-10} $. The 2-point Gaussian quadrature rule returns the integral of the black dashed curve, equal to ${\displaystyle u(-{\sqrt {\scriptstyle 1/3}})+u({\sqrt {\scriptstyle 1/3}})=2/3}$. Such a result is exact, since the green region has the same area as the red regions.*

References:

https://en.wikipedia.org/wiki/Gaussian_quadrature

http://mathworld.wolfram.com/GaussianQuadrature.html

Most applications of Gaussian quadrature involve computing the expectation of a function $u(.)$ of a continuous random variable $\tilde y$ with known probability density function $f(.)$

## Gauss-Legendre quadrature

Gaussian quadrature over a bounded interval with respect to the identity weight function,
$f(y) \equiv 1$, is called **Gauss-Legendre quadrature**. Gauss-Legendre quadrature is of special
interest because it is the Gaussian quadrature scheme appropriate for computing the area
under a curve. Gauss-Legendre quadrature is consistent for Riemann integrable functions.
That is, if $u(y)$ is Riemann integrable, then the approximation afforded by Gauss-Legendre
quadrature can be made arbitrarily precise by increasing the number of nodes $n$.




When the weight function $f(.)$ is the **probability density function** of some continuous random
variable $\tilde y$
, Gaussian quadrature has a very straightforward interpretation. In this
context, Gaussian quadrature essentially **"discretizes" the continuous random variable $\tilde y$**
by
replacing it with a discrete random variable with mass points $y_i$ and probabilities $f_i$ that
approximates $\tilde y$
in the sense that both random variables have the same moments of order
less than $2n$:

$$ \sum_{i=1}^{n} f_i y_i^k = E [\tilde y^k]   \thinspace \forall \thinspace k = 0,...,2n-1. $$


Given the mass points and probabilities of the **discrete approximant**, the expectation of any
function of the **continuous random variable $\tilde y$**
may be approximated using the expectation of
the function of the discrete approximant, which requires only the computation of a weighted
sum:

$$ E [u(\tilde y)] = \int_I  f(y)u(y) \thinspace dy \approx \sum_{i=1}^{n} f_i u(y_i)\thinspace .$$

## Probability distributions

1. Computing the $n$-degree Gaussian nodes and weights is a non-trivial task because it involves solving $2n$ nonlinear equations for ${y_i}$ and ${f_i}$. 

2. Effcient, specialized numerical routines for computing Gaussian quadrature nodes and weights are available for different weight functions, including virtually all the better known probability distributions such as the uniform, normal, gamma, exponential, Chi-square, and beta distributions.

3. All routines generate mass points $y_i$ and probabilities $f_i$ as output, and require the number of mass points $n$ as input, but differ with respect to other inputs.

In [1]:
import math
import numpy as np
import scipy.linalg as la
from scipy.special import gammaln
import sympy
sympy.init_printing()
from scipy import integrate
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def _qnwnorm1(n):
    """
    Compute nodes and weights for quadrature of univariate standard
    normal distribution
    Parameters
    ----------
    n : int
        The number of nodes
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        An n element array of nodes
    nodes : np.ndarray(dtype=float)
        An n element array of weights
    Notes
    -----
    Based of original function ``qnwnorm1`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    maxit = 100
    pim4 = 1 / np.pi**(0.25)
    m = np.fix((n + 1) / 2).astype(int)
    nodes = np.zeros(n)
    weights = np.zeros(n)

    for i in range(m):
        if i == 0:
            z = np.sqrt(2*n+1) - 1.85575 * ((2 * n + 1)**(-1 / 6.1))
        elif i == 1:
            z = z - 1.14 * (n ** 0.426) / z
        elif i == 2:
            z = 1.86 * z + 0.86 * nodes[0]
        elif i == 3:
            z = 1.91 * z + 0.91 * nodes[1]
        else:
            z = 2 * z + nodes[i-2]

        its = 0

        while its < maxit:
            its += 1
            p1 = pim4
            p2 = 0
            for j in range(1, n+1):
                p3 = p2
                p2 = p1
                p1 = z * math.sqrt(2.0/j) * p2 - math.sqrt((j - 1.0) / j) * p3

            pp = math.sqrt(2 * n) * p2
            z1 = z
            z = z1 - p1/pp
            if abs(z - z1) < 1e-14:
                break

        if its == maxit:
            raise ValueError("Failed to converge in _qnwnorm1")

        nodes[n - 1 - i] = z
        nodes[i] = -z
        weights[i] = 2 / (pp*pp)
        weights[n - 1 - i] = weights[i]

    weights /= math.sqrt(math.pi)
    nodes = nodes * math.sqrt(2.0)

    return nodes, weights

def _qnwbeta1(n, a=1.0, b=1.0):
    """
    Computes nodes and weights for quadrature on the beta distribution.
    Default is a=b=1 which is just a uniform distribution

    Parameters
    ----------
    n : scalar : int
        The number of quadrature points
    a : scalar : float, optional(default=1)
        First Beta distribution parameter
    b : scalar : float, optional(default=1)
        Second Beta distribution parameter
    Returns
    -------
    nodes : np.ndarray(dtype=float, ndim=1)
        The quadrature points
    weights : np.ndarray(dtype=float, ndim=1)
        The quadrature weights that correspond to nodes
    Notes
    -----
    Based of original function ``_qnwbeta1`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    # We subtract one and write a + 1 where we actually want a, and a
    # where we want a - 1
    a = a - 1
    b = b - 1

    maxiter = 25

    # Allocate empty space
    nodes = np.zeros(n)
    weights = np.zeros(n)

    # Find "reasonable" starting values.  Why these numbers?
    for i in range(n):
        if i == 0:
            an = a/n
            bn = b/n
            r1 = (1+a) * (2.78/(4+n*n) + .768*an/n)
            r2 = 1 + 1.48*an + .96*bn + .452*an*an + .83*an*bn
            z = 1 - r1/r2
        elif i == 1:
            r1 = (4.1+a) / ((1+a)*(1+0.156*a))
            r2 = 1 + 0.06 * (n-8) * (1+0.12*a)/n
            r3 = 1 + 0.012*b * (1+0.25*abs(a))/n
            z = z - (1-z) * r1 * r2 * r3
        elif i == 2:
            r1 = (1.67+0.28*a)/(1+0.37*a)
            r2 = 1+0.22*(n-8)/n
            r3 = 1+8*b/((6.28+b)*n*n)
            z = z-(nodes[0]-z)*r1*r2*r3
        elif i == n - 2:
            r1 = (1+0.235*b)/(0.766+0.119*b)
            r2 = 1/(1+0.639*(n-4)/(1+0.71*(n-4)))
            r3 = 1/(1+20*a/((7.5+a)*n*n))
            z = z+(z-nodes[-4])*r1*r2*r3
        elif i == n - 1:
            r1 = (1+0.37*b) / (1.67+0.28*b)
            r2 = 1 / (1+0.22*(n-8)/n)
            r3 = 1 / (1+8*a/((6.28+a)*n*n))
            z = z+(z-nodes[-3])*r1*r2*r3
        else:
            z = 3*nodes[i-1] - 3*nodes[i-2] + nodes[i-3]

        ab = a+b

        # Root finding
        its = 0
        z1 = -100
        while abs(z - z1) > 1e-10 and its < maxiter:
            temp = 2 + ab
            p1 = (a-b + temp*z)/2
            p2 = 1

            for j in range(2, n+1):
                p3 = p2
                p2 = p1
                temp = 2*j + ab
                aa = 2*j * (j+ab)*(temp-2)
                bb = (temp-1) * (a*a - b*b + temp*(temp-2) * z)
                c = 2 * (j - 1 + a) * (j - 1 + b) * temp
                p1 = (bb*p2 - c*p3)/aa

            pp = (n*(a-b-temp*z) * p1 + 2*(n+a)*(n+b)*p2)/(temp*(1 - z*z))
            z1 = z
            z = z1 - p1/pp

            if abs(z - z1) < 1e-12:
                break

            its += 1

        if its == maxiter:
            raise ValueError("Max Iteration reached.  Failed to converge")

        nodes[i] = z
        weights[i] = temp/(pp*p2)

    nodes = (1-nodes)/2
    weights = weights * math.exp(gammaln(a+n) + gammaln(b+n)
                                 - gammaln(n+1) - gammaln(n+ab+1))
    weights = weights / (2*math.exp(gammaln(a+1) + gammaln(b+1)
                         - gammaln(ab+2)))

    return nodes, weights


def _qnwgamma1(n, a=None):
    """
    Insert docs.  Default is a=0
    NOTE: For now I am just following compecon; would be much better to
    find a different way since I don't know what they are doing.
    Parameters
    ----------
    n : scalar : int
        The number of quadrature points
    a : scalar : float
        Gamma distribution parameter
    Returns
    -------
    nodes : np.ndarray(dtype=float, ndim=1)
        The quadrature points
    weights : np.ndarray(dtype=float, ndim=1)
        The quadrature weights that correspond to nodes
    Notes
    -----
    Based of original function ``qnwgamma1`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    if a is None:
        a = 0
    else:
        a -= 1

    maxit = 10

    factor = -math.exp(gammaln(a+n) - gammaln(n) - gammaln(a+1))
    nodes = np.zeros(n)
    weights = np.zeros(n)

    # Create nodes
    for i in range(n):
        # Reasonable starting values
        if i == 0:
            z = (1+a) * (3+0.92*a) / (1 + 2.4*n + 1.8*a)
        elif i == 1:
            z = z + (15 + 6.25*a) / (1 + 0.9*a + 2.5*n)
        else:
            j = i-1
            z = z + ((1 + 2.55*j) / (1.9*j) + 1.26*j*a / (1 + 3.5*j)) * \
                (z - nodes[j-1]) / (1 + 0.3*a)

        # root finding iterations
        its = 0
        z1 = -10000
        while abs(z - z1) > 1e-10 and its < maxit:
            p1 = 1.0
            p2 = 0.0
            for j in range(1, n+1):
                p3 = p2
                p2 = p1
                p1 = ((2*j - 1 + a - z)*p2 - (j - 1 + a)*p3) / j

            pp = (n*p1 - (n+a)*p2) / z
            z1 = z
            z = z1 - p1/pp
            its += 1

        if its == maxit:
            raise ValueError('Failure to converge')

        nodes[i] = z
        weights[i] = factor / (pp*n*p2)

    return nodes, weights

In [3]:
def qnwnorm(n, mu=None, sig2=None, usesqrtm=False):
    """
    Computes nodes and weights for multivariate normal distribution
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    mu : scalar or array_like(float), optional(default=zeros(d))
        The means of each dimension of the random variable. If a scalar
        is given, that constant is repeated d times, where d is the
        number of dimensions
    sig2 : array_like(float), optional(default=eye(d))
        A d x d array representing the variance-covariance matrix of the
        multivariate normal distribution.
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwnorm`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    n = np.asarray(n)
    d = n.size

    if mu is None:
        mu = np.zeros(d)
    else:
        mu = np.asarray(mu)

    if sig2 is None:
        sig2 = np.eye(d)
    else:
        sig2 = np.asarray(sig2).reshape(d, d)

    if all([x.size == 1 for x in [n, mu, sig2]]):
        nodes, weights = _qnwnorm1(n)
    else:
        nodes = []
        weights = []

        for i in range(d):
            _1d = _qnwnorm1(n[i])
            nodes.append(_1d[0])
            weights.append(_1d[1])

        nodes = gridmake(*nodes)
        weights = ckron(*weights[::-1])

    if usesqrtm:
        new_sig2 = la.sqrtm(sig2)
    else:  # cholesky
        new_sig2 = la.cholesky(sig2)

    if d > 1:
        nodes = nodes.dot(new_sig2) + mu  # Broadcast ok
    else:  # nodes.dot(sig) will not be aligned in scalar case.
        nodes = nodes * new_sig2 + mu

    return nodes.squeeze(), weights


def qnwlogn(n, mu=None, sig2=None):
    """
    Computes nodes and weights for multivariate lognormal distribution
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    mu : scalar or array_like(float), optional(default=zeros(d))
        The means of each dimension of the random variable. If a scalar
        is given, that constant is repeated d times, where d is the
        number of dimensions
    sig2 : array_like(float), optional(default=eye(d))
        A d x d array representing the variance-covariance matrix of the
        multivariate normal distribution.
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwlogn`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    nodes, weights = qnwnorm(n, mu, sig2)
    return np.exp(nodes), weights

def qnwunif(n, a, b):
    """
    Computes quadrature nodes and weights for multivariate uniform
    distribution
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    a : scalar or array_like(float)
        A length-d iterable of lower endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    b : scalar or array_like(float)
        A length-d iterable of upper endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwunif`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    n, a, b = list(map(np.asarray, [n, a, b]))
    nodes, weights = qnwlege(n, a, b)
    weights = weights / np.prod(b - a)
    return nodes, weights

def quadrect(f, n, a, b, kind='lege', *args, **kwargs):
    """
    Integrate the d-dimensional function f on a rectangle with lower and
    upper bound for dimension i defined by a[i] and b[i], respectively;
    using n[i] points.
    Parameters
    ----------
    f : function
        The function to integrate over. This should be a function
        that accepts as its first argument a matrix representing points
        along each dimension (each dimension is a column). Other
        arguments that need to be passed to the function are caught by
        `*args` and `**kwargs`
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    a : scalar or array_like(float)
        A length-d iterable of lower endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    b : scalar or array_like(float)
        A length-d iterable of upper endpoints. If a scalar is given,
        that constant is repeated d times, where d is the number of
        dimensions
    kind : string, optional(default='lege')
        Specifies which type of integration to perform. Valid
        values are:
        lege - Gauss-Legendre
        cheb - Gauss-Chebyshev
        trap - trapezoid rule
        simp - Simpson rule
        N    - Neiderreiter equidistributed sequence
        W    - Weyl equidistributed sequence
        H    - Haber  equidistributed sequence
        R    - Monte Carlo
    *args, **kwargs :
        Other arguments passed to the function f
    Returns
    -------
    out : scalar (float)
        The value of the integral on the region [a, b]
    Notes
    -----
    Based of original function ``quadrect`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    if kind.lower() == "lege":
        nodes, weights = qnwlege(n, a, b)
    elif kind.lower() == "cheb":
        nodes, weights = qnwcheb(n, a, b)
    elif kind.lower() == "trap":
        nodes, weights = qnwtrap(n, a, b)
    elif kind.lower() == "simp":
        nodes, weights = qnwsimp(n, a, b)
    else:
        nodes, weights = qnwequi(n, a, b, kind)

    out = weights.dot(f(nodes, *args, **kwargs))
    return out


def qnwbeta(n, a=1.0, b=1.0):
    """
    Computes nodes and weights for beta distribution
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    a : scalar or array_like(float), optional(default=1.0)
        A length-d
    b : array_like(float), optional(default=1.0)
        A d x d array representing the variance-covariance matrix of the
        multivariate normal distribution.
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwbeta`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    return _make_multidim_func(_qnwbeta1, n, a, b)


def qnwgamma(n, a=None):
    """
    Computes nodes and weights for gamma distribution
    Parameters
    ----------
    n : int or array_like(float)
        A length-d iterable of the number of nodes in each dimension
    mu : scalar or array_like(float), optional(default=zeros(d))
        The means of each dimension of the random variable. If a scalar
        is given, that constant is repeated d times, where d is the
        number of dimensions
    sig2 : array_like(float), optional(default=eye(d))
        A d x d array representing the variance-covariance matrix of the
        multivariate normal distribution.
    Returns
    -------
    nodes : np.ndarray(dtype=float)
        Quadrature nodes
    weights : np.ndarray(dtype=float)
        Weights for quadrature nodes
    Notes
    -----
    Based of original function ``qnwgamma`` in CompEcon toolbox by
    Miranda and Fackler
    References
    ----------
    Miranda, Mario J, and Paul L Fackler. Applied Computational
    Economics and Finance, MIT Press, 2002.
    """
    return _make_multidim_func(_qnwgamma1, n, a)

### How to Get the Quadrature Nodes (y) and Weights (w) for Common Distributions

-  For the normal distribution

y,w = qnwnorm(n,mu,var)
where, mu is the mean and var is the variance

-  For the lognormal distribution

y,w = qnwlogn(n,mu,var)

where, mu is the log mean and var is the log variance

-  For the Beta distribution

y,w = qnwbeta(n,a,b)

where, a and b are the shape parameters 

-  For the Gamma distribution

y,w = qnwgamma(n,a,b) 

where, a is the shape parameter and b is the scale parameter



### A Risk Problem
1. An agent’s utility of income exhibits constant absolute risk aversion $\alpha$:

$$u(y) = -exp(- \alpha  y)$$

2. The agent faces uncertain income $\tilde y$ that is log-normally distributed with parameters $\mu$ and $\sigma^2$

3. Would this agent accept a certain income $\bar y$ in place of his uncertain income $\tilde y$?

4. According to expected utility theory, yes, provided that

$$u(\bar y) \ge E[u(\tilde y)]$$

#### To compute the agent’s expected utility with random (or certain) income, execute:

In [4]:
#Number of nodes
#n = 3
n = 100

print("Case 1")
#Remark: the expected value of a lognormal random variable is equal to exp(mu+var/2)
#Case 1
mu = -0.5
var = 1

alpha = 2 #CARA
ybar = 1 #Certain income
y,w = qnwlogn(n,mu,var)

#Let's compute the expected income, given the parameter values
expectedy = w@y
print("The expected income in case 1 is:", expectedy)

#Let's compute the expected utility, given the parameter values
expectedutility = -w@np.exp(-alpha*y)
print("The expected utility in case 1 is:", expectedutility)

#Let's compute the utility of a given certain income
ucert = -np.exp(-alpha*ybar)
print("The utility of the (certain) average income in case 1 is:", ucert)

Case 1
The expected income in case 1 is: 0.9999999999999956
The expected utility in case 1 is: -0.18493376907531417
The utility of the (certain) average income in case 1 is: -0.1353352832366127


This generates, in case 1,  the approximation $Eu(\tilde y) = -0.1849$
which is less than $u(\bar y) = -0.135$


The agent would accept the certain income in case 1.

In [ ]:
#Let's compute the Certainty Equivalent income (ycert)
#It can be done by trial and error, or solving a non-linear equation in ycert
#ycert = 0.9
#ycert = 0.8
ycert = 0.8439
uycert = -np.exp(-alpha*ycert)
print("The certain income is:", ycert)
print("The utility of the certain income is:", uycert)

In [ ]:
#Case 2
print("Case 2")
mu = 0.2
var = 0.2

ynew,wnew = qnwlogn(n,mu,var)

#Let's compute the expected income, given the parameter values
expectedy = wnew@ynew
print("The expected income is:", expectedy)

#Let's compute the expected utility, given the parameter values
expectedutility = -wnew@np.exp(-alpha*ynew)
print("The expected utility in case 2 is:", expectedutility)

#Let's compute the utility of a given certain income
ucert = -np.exp(-alpha*ybar)
print("The utility of a (certain) income equal to 1 in case 2 is:", ucert)

This generates, in case 2,  the approximation $E[u(\tilde y)] = -0.1349$
which is higher than $u(\bar y=1) = -0.1353$


In case 2, the agent would not accept a certain income equal to 1 (but they would accept the new, higher, expected income).

In [ ]:
fig = plt.figure(figsize=(10,5))

plt.plot(y[0:68], w[0:68], label='Case 1, $\mu=-0.1$', linewidth = 1) 
plt.plot(ynew[0:66], wnew[0:66], label='Case 2, $\mu=0.2$', linewidth = 1, color = 'black') 

plt.legend(loc='best')
plt.xlabel('y')